<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-bert-tiny/source/lora-bert-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
if not os.path.exists('/content/drive/MyDrive/lora-bert-tiny/'):
    os.makedirs('/content/drive/MyDrive/lora-bert-tiny/')
output_dir = '/content/drive/MyDrive/lora-bert-tiny/'

In [13]:
!pip install datasets
!pip install evaluate
!pip install codecarbon
!pip freeze > /content/drive/MyDrive/lora-bert-tiny/requirements.txt

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [14]:
import torch
import torch.nn as nn
import numpy as np
from codecarbon import EmissionsTracker

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
import evaluate

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"])

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [16]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return {'total_parameters': parameters, 'trainable_parameters': trainable_parameters}

In [17]:
import csv

def write_results_to_csv(file_name, results):
  with open(file_name, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))
      writer.writeheader()
      for result in results:
        writer.writerow(result)

Number of times to run each section.

In [18]:
num_runs = 5

# Full Fine-Tuning



In [19]:
def fine_tune(epochs, batch_size, learning_rate):
  # Initialize result
  result = {"batch_size": size, "learning_rate": rate}

  # Track emissions
  tracker = EmissionsTracker('fine-tuning-bert-tiny', save_to_file=False)
  try:
    tracker.start()

    # Create the model
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

    # Setup the trainer
    training_args = TrainingArguments(
        "fine-tuning-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        disable_tqdm=True,
        report_to="none"
    )
    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    result |= trainer.train().metrics

    # Stop tracking emissions
    tracker.stop()

    # Store metrics and delete the tracker
    result |= trainer.evaluate() | get_trainable_parameters(model) | tracker.final_emissions_data.values
  finally:
    del tracker

  return result

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [20]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

In [12]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, set_seed
import time

ft_results = []
progress = 0
iterations = len(batch_sizes)*len(learning_rates)*num_runs

# Hyperparameter tuning
start_time = time.time()
for run in range(1, num_runs+1):
  set_seed(run)

  for size in batch_sizes:
    for rate in learning_rates:
      # Track progress
      progress += 1
      print(f"Progress: {progress}/{iterations}")

      # Fine-tune the model and store the results
      ft_result = {"run":run}
      ft_result |= fine_tune(epochs, size, rate)
      ft_results.append(ft_result)

  write_results_to_csv(output_dir + 'ft-results.csv', ft_results)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_minutes = runtime_seconds / 60

# Output the best result
max_ft_result = ft_results[0]
for _, result in enumerate(ft_results):
  if result['eval_f1'] > max_ft_result['eval_f1']:
    max_ft_result = result
print(f'\n================ \
        \nTotal Runtime: {runtime_minutes} minutes \
        \nBest Result: \
        \n\tF1={max_ft_result["eval_f1"]} \
        \n\tBatch Size={max_ft_result["batch_size"]} \
        \n\tLearning Rate={max_ft_result["learning_rate"]}'
)

[codecarbon INFO @ 00:11:52] [setup] RAM Tracking...
[codecarbon INFO @ 00:11:52] [setup] GPU Tracking...
[codecarbon INFO @ 00:11:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:11:52] [setup] CPU Tracking...
[codecarbon WARNING @ 00:11:52] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU



Progress: 1/100


[codecarbon WARNING @ 00:11:54] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:11:54] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:11:54] >>> Tracker's metadata:
[codecarbon INFO @ 00:11:54]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 00:11:54]   Python version: 3.10.12
[codecarbon INFO @ 00:11:54]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 00:11:54]   Available RAM : 50.994 GB
[codecarbon INFO @ 00:11:54]   CPU count: 8
[codecarbon INFO @ 00:11:54]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:11:54]   GPU count: 1
[codecarbon INFO @ 00:11:54]   GPU model: 1 x Tesla T4
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

{'eval_loss': 0.5478376746177673, 'eval_accuracy': 0.75, 'eval_f1': 0.840625, 'eval_runtime': 0.8275, 'eval_samples_per_second': 493.06, 'eval_steps_per_second': 61.633, 'epoch': 1.0}
{'loss': 0.6001, 'grad_norm': 6.086310863494873, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.590671181678772, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8124031007751938, 'eval_runtime': 0.7529, 'eval_samples_per_second': 541.931, 'eval_steps_per_second': 67.741, 'epoch': 2.0}
{'loss': 0.4803, 'grad_norm': 3.5907392501831055, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}


[codecarbon INFO @ 00:12:09] Energy consumed for RAM : 0.000080 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 00:12:09] Energy consumed for all GPUs : 0.000143 kWh. Total GPU Power : 34.40022061087558 W
[codecarbon INFO @ 00:12:09] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:12:09] 0.000400 kWh of electricity used since the beginning.


...

{'eval_loss': 0.5952083468437195, 'eval_accuracy': 0.6936274509803921, 'eval_f1': 0.8169838945827232, 'eval_runtime': 0.6042, 'eval_samples_per_second': 675.252, 'eval_steps_per_second': 6.62, 'epoch': 4.0}
Progress: 100/100


[codecarbon WARNING @ 00:36:28] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:36:28] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:36:28] >>> Tracker's metadata:
[codecarbon INFO @ 00:36:28]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 00:36:28]   Python version: 3.10.12
[codecarbon INFO @ 00:36:28]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 00:36:28]   Available RAM : 50.994 GB
[codecarbon INFO @ 00:36:28]   CPU count: 8
[codecarbon INFO @ 00:36:28]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:36:28]   GPU count: 1
[codecarbon INFO @ 00:36:28]   GPU model: 1 x Tesla T4
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

{'eval_loss': 0.6439599990844727, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.6122, 'eval_samples_per_second': 666.492, 'eval_steps_per_second': 6.534, 'epoch': 1.0}
{'eval_loss': 0.6185536980628967, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5996, 'eval_samples_per_second': 680.405, 'eval_steps_per_second': 6.671, 'epoch': 2.0}
{'eval_loss': 0.6116704344749451, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5951, 'eval_samples_per_second': 685.605, 'eval_steps_per_second': 6.722, 'epoch': 3.0}


[codecarbon INFO @ 00:36:36] Energy consumed for RAM : 0.000039 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 00:36:36] Energy consumed for all GPUs : 0.000111 kWh. Total GPU Power : 53.62237985736155 W
[codecarbon INFO @ 00:36:36] Energy consumed for all CPUs : 0.000088 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:36:36] 0.000238 kWh of electricity used since the beginning.


{'eval_loss': 0.609874427318573, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5996, 'eval_samples_per_second': 680.485, 'eval_steps_per_second': 6.671, 'epoch': 4.0}
{'train_runtime': 6.8018, 'train_samples_per_second': 2157.088, 'train_steps_per_second': 17.054, 'train_loss': 0.6398460125101024, 'epoch': 4.0}
{'eval_loss': 0.609874427318573, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5987, 'eval_samples_per_second': 681.5, 'eval_steps_per_second': 6.681, 'epoch': 4.0}

Total Runtime: 24.73579385280609 minutes         
Best Result:         
	F1=0.8386075949367089         
	Batch Size=64         
	Learning Rate=0.0003


# Low-Rank Adaptation (LoRA)

In [ ]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

In [ ]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [ ]:
def configure_lora_model(model, rank, alpha):
  # Replace the query and value linear layers with LoRA layers
  for _, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  # Freeze the pre-trained weights
  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False

In [ ]:
def fine_tune_with_lora_model(model, rank, alpha, epochs, batch_size, learning_rate):
  # Initialize result
  result = {"rank": rank, "alpha": alpha, "batch_size": batch_size, "learning_rate": learning_rate}

  # Track emissions
  tracker = EmissionsTracker('lora-bert-tiny', save_to_file=False)
  try:
    tracker.start()

    # Configure an existing model with new LoRA layers
    configure_lora_model(model, rank, alpha)

    # Setup the trainer
    training_args = TrainingArguments(
        "lora-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        disable_tqdm=True,
        report_to="none"
    )
    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    result |= trainer.train().metrics

    # Stop tracking emissions
    tracker.stop()

    # Store metrics and delete the tracker
    result |= trainer.evaluate() | get_trainable_parameters(model) | tracker.final_emissions_data.values
  finally:
    del tracker

  return result

In [17]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [ ]:
from transformers import Trainer, TrainingArguments, set_seed
import time

lora_results = []
progress = 0
iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)*num_runs

# Hyperparameter tuning
start_time = time.time()
for run in range(1, num_runs+1):
  set_seed(run)

  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          # Track progress
          progress += 1
          print(f"Progress: {progress}/{iterations}")

          # Fine-tune the LoRA model and store the results
          lora_result = {"run": run}
          lora_result |= fine_tune_with_lora_model(lora_model, rank, alpha, epochs, size, rate)
          lora_results.append(lora_result)

  write_results_to_csv(output_dir + 'lora-results.csv', lora_results)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_hours = runtime_seconds / 3600

# Output the best result
max_lora_result = lora_results[0]
for _, result in enumerate(lora_results):
  if result['eval_f1'] > max_lora_result['eval_f1']:
    max_lora_result = result
print(f'\n================ \
        \nTotal Runtime: {runtime_hours} hours \
        \nBest Result: \
        \n\tF1={max_lora_result["eval_f1"]} \
        \n\tRank={max_lora_result["rank"]} \
        \n\tAlpha={max_lora_result["alpha"]} \
        \n\tBatch Size={max_lora_result["batch_size"]} \
        \n\tLearning Rate={max_lora_result["learning_rate"]}'
)

[codecarbon INFO @ 00:36:37] [setup] RAM Tracking...
[codecarbon INFO @ 00:36:37] [setup] GPU Tracking...
[codecarbon INFO @ 00:36:37] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:36:37] [setup] CPU Tracking...
[codecarbon WARNING @ 00:36:37] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU



Progress: 1/2500


[codecarbon WARNING @ 00:36:38] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:36:38] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:36:38] >>> Tracker's metadata:
[codecarbon INFO @ 00:36:38]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 00:36:38]   Python version: 3.10.12
[codecarbon INFO @ 00:36:38]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 00:36:38]   Available RAM : 50.994 GB
[codecarbon INFO @ 00:36:38]   CPU count: 8
[codecarbon INFO @ 00:36:38]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:36:38]   GPU count: 1
[codecarbon INFO @ 00:36:38]   GPU model: 1 x Tesla T4


{'eval_loss': 0.6247038841247559, 'eval_accuracy': 0.6813725490196079, 'eval_f1': 0.8099415204678363, 'eval_runtime': 0.7752, 'eval_samples_per_second': 526.32, 'eval_steps_per_second': 65.79, 'epoch': 1.0}
{'loss': 0.6576, 'grad_norm': 0.5037346482276917, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}
{'eval_loss': 0.6202483773231506, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8116788321167884, 'eval_runtime': 0.7992, 'eval_samples_per_second': 510.494, 'eval_steps_per_second': 63.812, 'epoch': 2.0}
{'loss': 0.6277, 'grad_norm': 0.49709364771842957, 'learning_rate': 0.00013660130718954247, 'epoch': 2.178649237472767}


[codecarbon INFO @ 00:36:53] Energy consumed for RAM : 0.000080 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 00:36:53] Energy consumed for all GPUs : 0.000144 kWh. Total GPU Power : 34.5985816934756 W
[codecarbon INFO @ 00:36:53] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:36:53] 0.000401 kWh of electricity used since the beginning.


...

{'eval_loss': 0.6855593919754028, 'eval_accuracy': 0.5759803921568627, 'eval_f1': 0.6587771203155819, 'eval_runtime': 0.6235, 'eval_samples_per_second': 654.386, 'eval_steps_per_second': 11.227, 'epoch': 4.0}
{'train_runtime': 7.0668, 'train_samples_per_second': 2076.191, 'train_steps_per_second': 32.83, 'train_loss': 0.7082098599137931, 'epoch': 4.0}


[codecarbon INFO @ 00:54:09] [setup] RAM Tracking...
[codecarbon INFO @ 00:54:09] [setup] GPU Tracking...
[codecarbon INFO @ 00:54:09] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:54:09] [setup] CPU Tracking...
[codecarbon WARNING @ 00:54:09] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU



{'eval_loss': 0.6855593919754028, 'eval_accuracy': 0.5759803921568627, 'eval_f1': 0.6587771203155819, 'eval_runtime': 0.6245, 'eval_samples_per_second': 653.281, 'eval_steps_per_second': 11.208, 'epoch': 4.0}
Progress: 77/2500


[codecarbon WARNING @ 00:54:10] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:54:10] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:54:10] >>> Tracker's metadata:
[codecarbon INFO @ 00:54:10]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 00:54:10]   Python version: 3.10.12
[codecarbon INFO @ 00:54:10]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 00:54:10]   Available RAM : 50.994 GB
[codecarbon INFO @ 00:54:10]   CPU count: 8
[codecarbon INFO @ 00:54:10]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 00:54:10]   GPU count: 1
[codecarbon INFO @ 00:54:10]   GPU model: 1 x Tesla T4


{'eval_loss': 0.6343888640403748, 'eval_accuracy': 0.678921568627451, 'eval_f1': 0.8035982008995503, 'eval_runtime': 0.5949, 'eval_samples_per_second': 685.809, 'eval_steps_per_second': 6.724, 'epoch': 1.0}
{'eval_loss': 0.6240852475166321, 'eval_accuracy': 0.6887254901960784, 'eval_f1': 0.8135095447870778, 'eval_runtime': 0.593, 'eval_samples_per_second': 688.009, 'eval_steps_per_second': 6.745, 'epoch': 2.0}
{'eval_loss': 0.6218690276145935, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8100147275405007, 'eval_runtime': 0.6113, 'eval_samples_per_second': 667.44, 'eval_steps_per_second': 6.544, 'epoch': 3.0}
